In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../')

print(sys.path)

['/home/tiendat/transformer-entropy-ids/notebooks', '/home/tiendat/miniconda3/envs/torchtf/lib/python39.zip', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/lib-dynload', '', '/home/tiendat/miniconda3/envs/torchtf/lib/python3.9/site-packages', '../']


In [2]:
import torch
import torch.nn as nn
import math
import argparse
from dataPreprocess import DatasetPreprocess
from torch.utils.data import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, classification_report, confusion_matrix
import os
import torch.optim as optim
import time
import warnings
import torch.nn.functional as F
import time
from torchsummary import summary

In [3]:
def draw_confusion(label_y, pre_y, path):
    confusion = confusion_matrix(label_y, pre_y)
    print(confusion)
    return confusion

def write_result(label_y, pre_y, classes_num):
    if classes_num > 2:
        accuracy = accuracy_score(label_y, pre_y)
        macro_precision = precision_score(label_y, pre_y, average='macro')
        macro_recall = recall_score(label_y, pre_y, average='macro')
        macro_f1 = f1_score(label_y, pre_y, average='macro')
        micro_precision = precision_score(label_y, pre_y, average='micro')
        micro_recall = recall_score(label_y, pre_y, average='micro')
        micro_f1 = f1_score(label_y, pre_y, average='micro')
        print('  -- test result: ')
        print('    -- accuracy: ', accuracy)
        print('    -- macro precision: ', macro_precision)
        print('    -- macro recall: ', macro_recall)
        print('    -- macro f1 score: ', macro_f1)
        print('    -- micro precision: ', micro_precision)
        print('    -- micro recall: ', micro_recall)
        print('    -- micro f1 score: ', micro_f1)
        report = classification_report(label_y, pre_y)
        print(report)
    else:
        accuracy = accuracy_score(label_y, pre_y)
        precision = precision_score(label_y, pre_y)
        recall = recall_score(label_y, pre_y)
        f1 = f1_score(label_y, pre_y)
        print('  -- test result: ')
        print('    -- accuracy: ', accuracy)
        print('    -- recall: ', recall)
        print('    -- precision: ', precision)
        print('    -- f1 score: ', f1)
        report = classification_report(label_y, pre_y)
        print(report)

In [6]:
seed = 1
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

#================================================================
root_dir = "../road/predict_fab_multi/TFRecord_w15_s15/1/"
window_size = 15
d_model = 12
max_time_position = 10000
gran = 1e-6
log_e = 2
batch_size = 10

train_dataset = DatasetPreprocess(root_dir, window_size, window_size, d_model, max_time_position, gran, log_e, is_train=True)
test_dataset = DatasetPreprocess(root_dir, window_size,window_size, d_model, max_time_position, gran, log_e, is_train=False)


print("TRAIN SIZE:", len(train_dataset), " TEST SIZE:", len(test_dataset), " SIZE:", len(train_dataset)+len(test_dataset), " TRAIN RATIO:", round(len(train_dataset)/(len(train_dataset)+len(test_dataset))*100), "%")

# 2 DataLoader? 
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)
print('finish load data')


IndentationError: unexpected indent (2522242971.py, line 26)

In [ ]:
class Config:
    def __init__(self, args):
        self.model_name = 'IDS-Transformer_' + args.type
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        
        self.payload_size = 8 
            
        if args.mode == "ae":
            self.mode = args.mode
            self.dout_payload = 7 
            self.dout_header = 3 
            self.d_model = self.dout_payload + self.dout_header # dimension input of transformer encoder
            self.nhead = 10 # ori: 5
        elif args.mode == "dnn":
            self.mode = args.mode
            self.dout_payload = 32
            self.dout_header = 8 
            self.d_model = self.dout_payload + self.dout_header 
            self.nhead = 5 # ori: 5
        elif args.mode == "cb":
            self.mode = args.mode
            self.dout_mess = 10
            self.d_model = self.dout_mess
            self.nhead = 10 # ori: 5
        else:
            self.mode = args.mode
            self.dout_mess = 12
            self.d_model = self.dout_mess
            self.nhead = 6 # ori: 5
        
        self.tse = args.tse
        self.pad_size = args.window_size # 15
        self.window_size =  args.window_size # 15
        self.max_time_position = 10000
        self.num_layers = 6
        self.gran = 1e-6
        self.log_e = 2
        
        if args.type == 'chd':
            self.classes_num = 5
        elif args.type == 'road_mas':
            self.classes_num = 6
        else: # road_fab
            self.classes_num = 7 
        
            
        self.batch_size = args.batch_size
        self.epoch_num = args.epoch
        self.lr = args.lr #0.00001 learning rate 
        self.root_dir = args.indir
        
        # self.root_dir = './data/Processed/TFRecord_w29_s29/2/'
        # self.root_dir = './road/predict_fab_multi/TFRecord_w15_s15/1/'
        # self.root_dir = './road/predict_mas/TFRecord_w15_s15/4/'
        self.model_save_path = '../model/' + self.model_name + '/'
        if not os.path.exists(self.model_save_path):
            os.mkdir(self.model_save_path)
        self.result_file = '/home/tiendat/transformer-entropy-ids/result/trans8_performance.txt'

        self.isload_model = False  
        self.start_epoch = 24  # The epoch of the loaded model
        self.model_path = 'model/' + self.model_name + '/' + self.model_name + '_model_' + str(self.start_epoch) + '.pth' 

class CosineWarmupScheduler(optim.lr_scheduler._LRScheduler):
    def __init__(self, optimizer, warmup, max_iters):
        self.warmup = warmup
        self.max_num_iters = max_iters
        super().__init__(optimizer)

    def get_lr(self):
        lr_factor = self.get_lr_factor(epoch=self.last_epoch)
        return [base_lr * lr_factor for base_lr in self.base_lrs]

    def get_lr_factor(self, epoch):
        lr_factor = 0.5 * (1 + np.cos(np.pi * epoch / self.max_num_iters))
        if epoch <= self.warmup:
            lr_factor *= epoch * 1.0 / self.warmup
        return lr_factor
    
class Autoencoder1D(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(Autoencoder1D, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, output_dim)
        )
    
    def forward(self, x):
        x = self.encoder(x)
        return F.relu(x)
    
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class DNN(nn.Module):
    def __init__(self, d_in, d_out):  # config.payload_size, config.dout_payload
        super(DNN, self).__init__()
        self.l1 = nn.Linear(d_in, 128)
        self.l2 = nn.Linear(128, 64)
        self.l3 = nn.Linear(64, d_out)

    def forward(self, x):
        #print('x: ', x.cpu().numpy()[0])
        out = F.relu(self.l1(x))
        out = F.relu(self.l2(out))
        out = F.relu(self.l3(out))
        #print('dnn out: ', out.cpu().detach().numpy()[0])
        return out

class Time_Positional_Encoding(nn.Module):
    def __init__(self, embed, max_time_position, device):
        super(Time_Positional_Encoding, self).__init__()
        self.device = device

    def forward(self, x, time_position):
        out = x.permute(1, 0, 2)
        out = out + nn.Parameter(time_position, requires_grad=False).to(self.device)
        out = out.permute(1, 0, 2)
        return out
    
class TransformerPredictor(nn.Module):
    def __init__(self, config):
        super(TransformerPredictor, self).__init__()
        
        self.ae = Autoencoder1D(12, config.dout_mess).to(config.device)
        self.pad_size = config.pad_size
        
        self.position_embedding = Time_Positional_Encoding(config.d_model, config.max_time_position, config.device).to(config.device)
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=config.d_model, nhead=config.nhead).to(config.device)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=config.num_layers).to(config.device)
        self.fc = nn.Linear(config.d_model, config.classes_num).to(config.device)
        
    def forward(self, header, sl_sum, mask, time_position):
        x = torch.concat((header, sl_sum), dim=-1)
        ae_out = torch.empty((x.shape[0], 10, 0)).to(config.device)
        for i in range(self.pad_size):
            tmp = self.ae(x[:, i, :]).unsqueeze(2)
            ae_out = torch.concat((ae_out, tmp), dim=2)
        x = ae_out.permute(2, 0, 1)
            
        out = self.position_embedding(x, time_position)
            
        out = self.transformer_encoder(out, src_key_padding_mask=mask)
        out = out.permute(1, 0, 2)
        out = torch.sum(out, 1)
        out = self.fc(out)
        return out

args = argparse.Namespace(
    indir="",
    window_size=15,
    type="chd",
    epoch=200,
    batch_size=32,
    lr=0.0001,
    mode="cb",
    tse=True
)

config = Config(args)
model = TransformerPredictor(config)
start_epoch = -1
loss_func = nn.CrossEntropyLoss().to(config.device)
opt = optim.Adam(model.parameters(), lr=config.lr)
lr_scheduler = CosineWarmupScheduler(opt, warmup=500, max_iters=200)

In [ ]:
import time
import torch

# Assuming 'inputs' is your input data
inputs = [torch.randn(10, 15, 4), torch.randn(10, 15, 8), torch.randn(10, 15), torch.randn(10, 15, 12)]

# Move the model and inputs to the same device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
inputs = [i.to(device) for i in inputs]

# Make sure the model is in evaluation mode
model.eval()

# Warm up (for more accurate timing)
with torch.no_grad():
    _ = model(*inputs)

# Time the inference
start_time = time.time()
with torch.no_grad():
    _ = model(*inputs)
end_time = time.time()

print(f'Inference time: {end_time - start_time} seconds')

In [7]:
summary(model, [(10, 15, 4), (10, 15, 8), (10, 15,), (10, 15, 12)], batch_dim=None)

Layer (type:depth-idx)                        Output Shape              Param #
├─Time_Positional_Encoding: 1-1               [15, 10, 12]              --
├─TransformerEncoder: 1-2                     [15, 10, 12]              --
|    └─ModuleList: 2                          []                        --
|    |    └─TransformerEncoderLayer: 3-1      [15, 10, 12]              51,884
|    |    └─TransformerEncoderLayer: 3-2      [15, 10, 12]              51,884
|    |    └─TransformerEncoderLayer: 3-3      [15, 10, 12]              51,884
|    |    └─TransformerEncoderLayer: 3-4      [15, 10, 12]              51,884
|    |    └─TransformerEncoderLayer: 3-5      [15, 10, 12]              51,884
├─Linear: 1-3                                 [10, 7]                   91
Total params: 259,511
Trainable params: 259,511
Non-trainable params: 0
Total mult-adds (M): 0.75
Input size (MB): 0.01
Forward/backward pass size (MB): 11.93
Params size (MB): 0.99
Estimated Total Size (MB): 12.93


Layer (type:depth-idx)                        Output Shape              Param #
├─Time_Positional_Encoding: 1-1               [15, 10, 12]              --
├─TransformerEncoder: 1-2                     [15, 10, 12]              --
|    └─ModuleList: 2                          []                        --
|    |    └─TransformerEncoderLayer: 3-1      [15, 10, 12]              51,884
|    |    └─TransformerEncoderLayer: 3-2      [15, 10, 12]              51,884
|    |    └─TransformerEncoderLayer: 3-3      [15, 10, 12]              51,884
|    |    └─TransformerEncoderLayer: 3-4      [15, 10, 12]              51,884
|    |    └─TransformerEncoderLayer: 3-5      [15, 10, 12]              51,884
├─Linear: 1-3                                 [10, 7]                   91
Total params: 259,511
Trainable params: 259,511
Non-trainable params: 0
Total mult-adds (M): 0.75
Input size (MB): 0.01
Forward/backward pass size (MB): 11.93
Params size (MB): 0.99
Estimated Total Size (MB): 12.93

In [25]:
from thop import profile

x = torch.rand((10, 15, 4)).to(config.device)
y = torch.rand((10, 15, 8)).to(config.device)
z = torch.rand((10, 15,)).to(config.device)
a = torch.rand((10, 15, 10)).to(config.device)

macs, params = profile(model, inputs=[x, y, z, a])
print('MACs (G): ', macs/1000**2)
print('Params (M): ', params/1000**2)

[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.normalization.LayerNorm'>.
MACs (G):  37.0949
Params (M):  0.259517


In [73]:
for epoch in range(start_epoch + 1, config.epoch_num):
        fin = open(config.result_file, 'a')
        print('--- epoch ', epoch)
        fin.write('-- epoch ' + str(epoch) + '\n')
        for i, sample_batch in enumerate(train_loader):
            batch_header = sample_batch['header'].type(torch.FloatTensor).to(config.device)
            batch_payload = sample_batch['payload'].type(torch.FloatTensor).to(config.device)
            batch_mask = sample_batch['mask'].to(config.device)
            batch_label = sample_batch['label'].to(config.device)
            batch_time_position = sample_batch['time'].to(config.device)
            
            out = model(batch_header, batch_payload, batch_mask, batch_time_position)
            loss = loss_func(out, batch_label)
            opt.zero_grad()
            loss.backward()
            opt.step()
            lr_scheduler.step()
            aa
        # test
        label_y = []
        pre_y = []
        with torch.no_grad():
            for j, test_sample_batch in enumerate(test_loader):
                test_header = test_sample_batch['header'].type(torch.FloatTensor).to(config.device)
                test_payload = test_sample_batch['payload'].type(torch.FloatTensor).to(config.device)
                test_mask = test_sample_batch['mask'].to(config.device)
                test_label = test_sample_batch['label'].to(config.device)
                test_time_position = test_sample_batch['time'].to(config.device)
                
                test_out = model(test_header, test_payload, test_mask, test_time_position)

                pre = torch.max(test_out, 1)[1].cpu().numpy()
                
                pre_y = np.concatenate([pre_y, pre], 0)
                label_y = np.concatenate([label_y, test_label.cpu().numpy()], 0)
            write_result(label_y, pre_y, config.classes_num)
            draw_confusion(label_y, pre_y, '')

--- epoch  0
torch.Size([10, 15, 4])
torch.Size([10, 15, 8])
torch.Size([10, 15])
torch.Size([10, 15, 12])
torch.Size([15, 10, 12])


NameError: name 'aa' is not defined

In [5]:
def evaluate(confusion_matrix):
    tn, fp, fn, tp = confusion_matrix.ravel()
    fnr = fn/(tp + fn)
    accu = (tp + tn) / (tp + tn + fp + fn)
    err = (fn + fp) / (tp + tn + fp + fn)
    precision = tp/(tp + fp)
    recall = 1 - fnr
    f1score = (2 * precision * recall) / (precision + recall)
    print(tn, fp)
    print(fn, tp)
    print('Accuracy: ', accu)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 score: ', f1score)
    print('Error rate: ', err)
    print('False negative rate: ', fnr)

In [6]:

evaluate(np.array([[32361,430],[2,52908]]))

32361 430
2 52908
Accuracy:  0.9949592186788953
Precision:  0.9919382054070269
Recall:  0.9999621999621999
F1 score:  0.9959340411113621
Error rate:  0.0050407813211047715
False negative rate:  3.78000378000378e-05


In [ ]:

draw_confusion(label_y, pre_y, '')

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

# Data for confusion matrix
confusion_matrix_chd = np.array([[150833,      0,      0,      0,      0],
                             [     0,  11236,      0,      0,      0],
                             [     0,      0,  13346,      0,      0],
                             [     0,      0,      0,  19585,      0],
                             [     0,      0,      0,      0,  21412]])
confusion_matrix_chd1 = np.array([[150833,      0,      0,      0,      0],
                             [     12,  11224,      0,      0,      0],
                             [     0,      0,  13336,      10,      0],
                             [     13,      0,      0,  19572,      0],
                             [     0,      2,      0,      0,  21410]])

confusion_matrix_2 = np.array([
    [15102, 0, 0, 1, 8, 0, 0],
    [0, 9, 0, 0, 0, 0, 0],
    [0, 0, 12, 0, 0, 0, 0],
    [0, 0, 0, 2288, 0, 0, 0],
    [2, 0, 0, 0, 1689, 0, 0],
    [0, 0, 0, 0, 0, 1071, 0],
    [0, 0, 0, 0, 0, 0, 1006]
])

cm_mas = np.array([
    [15102,0,0,0,0,0],
    [0,9,0,0,0,0],
    [0,0,2250,0,0,0],
    [0,0,0,1587,0,0],
    [0,0,0,0,1051,0],
    [0,0,0,0,0,990]
])

# Labels for classes
labels = ["Normal", "DoS", "Fuzz", "Gear", "RPM"]

labelss = ["Normal", "Attack"]
labelsss = ['Normal', 'MEC', 'Fuzzing', 'MS', 
              'RLOn', 'RLOff', 'CS']
labels_mas = ['Normal', 'MEC', 'MS', 
              'RLOn', 'RLOff', 'CS']
cm = np.array([[14394, 0],
               [0, 6448]])

fig, axs = plt.subplots(ncols=2, figsize=(25,11))

sns.heatmap(confusion_matrix_chd, annot=True, fmt="d", cmap="Greys", xticklabels=labels, yticklabels=labels, annot_kws={"size": 16}, ax=axs[0])
axs[0].tick_params(labelsize=18)
axs[0].set_ylabel('True Label', fontsize=20, labelpad=15)
axs[0].set_xlabel('Predicted Label', fontsize=20, labelpad=15)
axs[0].set_title('Ours', fontsize=20)

sns.heatmap(confusion_matrix_chd1, annot=True, fmt="d", cmap="Greys", xticklabels=labels, yticklabels=labels, annot_kws={"size": 16}, ax=axs[1])
axs[1].tick_params(labelsize=18)
axs[1].set_ylabel('True Label', fontsize=20, labelpad=15)
axs[1].set_xlabel('Predicted Label', fontsize=20, labelpad=15)
axs[1].set_title('Traditional transformer', fontsize=20)

plt.tight_layout()
plt.show()


NameError: name 'model' is not defined

In [4]:
import numpy as np

# Confusion matrix
confusion_matrix_2 = np.array([
    [15102,0,0,1,8,5],
    [7,2,0,0,0,0],
    [66,0,2181,3,0,0],
    [   46,0,1,1540,0,0],
    [   20,0,0,0,1031,0],
    [    3,0,0,0,0,987],
])
confusion_matrix = np.array([
    [15102,0,0,1,8,0,0],
    [0,9,0,0,0,0,0],
    [0,0,12,0,0,0,0],
    [0,0,0,2288,0,0,0],
    [2,0,0,0,1689,0,0],
    [0,0,0,0,0,1071,0],
    [0,0,0,0,0,0,1006],
])
confusion_matrix_test = np.array([
    [15094,1,0,0,11,5,0],
    [0,9,0,0,0,0,0],
    [0,0,12,0,0,0,0],
    [7,0,0,2281,0,0,0],
    [17,0,0,0,1673,1,0],
    [0,0,0,0,0,1071,0],
    [0,0,0,0,0,0,1006],
])


# List of attributes
attributes = ['Normal', 'max_engine_coolant_temp_attack', 'fuzzing_attack', 'max_speedometer_attack', 
              'reverse_light_on_attack', 'reverse_light_off_attack', 'correlated_signal_attack']
attributes2 = ['Normal', 'max_engine_coolant_temp_attack', 'max_speedometer_attack', 
              'reverse_light_on_attack', 'reverse_light_off_attack', 'correlated_signal_attack']

# F1 score calculation for each attribute
f1_scores = []

for i in range(len(attributes)):
    TP = confusion_matrix[i, i]
    TN = sum(sum(confusion_matrix)) - TP
    FP = sum(confusion_matrix[:, i]) - TP
    FN = sum(confusion_matrix[i, :]) - TP
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1_score = 2 * precision * recall / (precision + recall)
    FNR = (FN / (TP + FN))*100  # False Negative Rate
    FPR = (FP / (FP + TN))*100  # False Positive Rate
    print('Attribute: ', attributes[i])
    print('Accuracy: ', accuracy)
    print('Precision: ', precision)
    print('Recall: ', recall)
    print('F1 score: ', f1_score)
    print('False Negative Rate: ', FNR)  # Print FNR
    print('False Positive Rate: ', FPR)  # Print FPR
    f1_scores.append(f1_score)

f1_scores_per_attribute = dict(zip(attributes, f1_scores))
f1_scores_per_attribute


NameError: name 'model' is not defined